In [ ]:
#| default_exp visualization

In [ ]:
#| export
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
#| export

def plot_heatmap(matrix, 
                 title="Expectation of Memory Traces", 
                 xlabel="Trace", 
                 ylabel="Time Step", 
                 color="Blues", 
                 extent = None, 
                 figsize = (8,6),
                 xticks = None,
                 xtick_labels = None,
                 ax = None):
    """
    Plots a heatmap of the given matrix using the intensity of a single color.
    
    :param matrix: 2D numpy array to be visualized.
    :param title: Title of the heatmap.
    :param xlabel: Label for the x-axis.
    :param ylabel: Label for the y-axis.
    :param color: The base color for intensity visualization.
    """
    if ax is None:
        plt.figure(figsize=figsize)
    else:
        plt.sca(ax)  # Set current axis
    if extent is None:
        extent = [0,np.shape(matrix)[1],np.shape(matrix)[0], 0]
    plt.imshow(matrix, cmap=plt.colormaps.get_cmap(color), aspect='auto', extent = extent)
    plt.colorbar(label="Intensity")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if not xticks is None:
        if xtick_labels is None:
            plt.xticks(xticks)
        else:
            plt.xticks(xticks, xtick_labels)
    if ax is None:
        plt.show()

In [ ]:
#| export

def simulation_boxplot_summary(data, 
                               param1_values, 
                               param2_values, 
                               title, 
                               ylabel, 
                               parameter_names = ['Paramter 1', 'Parameter 2']):
    '''
    data: 3D array, with simulation results. First two dimensions give value of a simulation parameter, third dimension gives the simulation number.
    '''


    
    A = np.shape(data)[0] #param1
    B = np.shape(data)[1] #param2
    C = np.shape(data)[2] #simulations
    # Define colors for B indices
    colors = plt.cm.viridis(np.linspace(0, 1, B))  # B distinct colors
    
    d = 3 #priming softmax index
    for b in range(B):
        box_data = [data[a, b, :] for a in range(A)]
        positions = np.arange(A) + (b - B / 2) * 0.1  # Adjust positions to avoid overlap
        plt.boxplot(box_data, positions=positions, widths=0.1, patch_artist=True, 
                    boxprops=dict(facecolor=colors[b], alpha=0.6), 
                    medianprops=dict(color='black'))
        # Calculate and add 95% credible interval error bars
        for a in range(A):
            mean_val = np.mean(data[a, b, :])
            ci_low, ci_high = stats.t.interval(0.95, df=C-1, loc=mean_val, scale=stats.sem(data[a, b, :]))
            plt.errorbar(positions[a], mean_val, yerr=[[mean_val - ci_low], [ci_high - mean_val]], fmt='o', color='black')
    
    # Create legend
    legend_patches = [plt.Line2D([0], [0], color=colors[b], lw=4, label=f'{parameter_names[1]}={param2_values[b]}') for b in range(B)]
    plt.legend(handles=legend_patches, loc = 2)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(parameter_names[0])
    plt.xticks(range(A), labels=[f"{param1_values[a]}" for a in range(A)])